In [1]:
import os
os.getcwd()

'D:\\Documents\\PycharmProjects\\Ookla-speed'

Must run from Conda venv for the geo-packages to work 

Example from ookla-open-data tutorials on https://github.com/teamookla/ookla-open-data/blob/master/tutorials/aggregate_by_county_py.ipynb

In [2]:
#Geopandas required all the following dependencies: pandas, shapely, fiona, and pyproj. Optional dependencies includes matplotlib, rtree
!pip install shapely pyproj rtree matplotlib pandas fiona

Issue with Fiona and other packages can be download here for Window https://www.lfd.uci.edu/~gohlke/pythonlibs/

In [3]:
!pip install geopandas

In [4]:
!pip install adjustText

In [5]:
%matplotlib inline

from datetime import datetime

import geopandas as gp
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from shapely.geometry import Point
from adjustText import adjust_text

Functions to download the datasets 

In [6]:
def quarter_start(year: int, q: int) -> datetime:
    if not 1 <= q <= 4:
        raise ValueError("Quarter must be within [1, 2, 3, 4]")

    month = [1, 4, 7, 10]
    return datetime(year, month[q - 1], 1)


def get_tile_url(service_type: str, year: int, q: int) -> str:
    dt = quarter_start(year, q)

    base_url = "https://ookla-open-data.s3-us-west-2.amazonaws.com/shapefiles/performance"
    url = f"{base_url}/type%3D{service_type}/year%3D{dt:%Y}/quarter%3D{q}/{dt:%Y-%m-%d}_performance_fixed_tiles.zip"
    return url

In [7]:
tile_url = get_tile_url("fixed", 2020, 2)
tile_url

'https://ookla-open-data.s3-us-west-2.amazonaws.com/shapefiles/performance/type%3Dfixed/year%3D2020/quarter%3D2/2020-04-01_performance_fixed_tiles.zip'

In [8]:
tiles = gp.read_file(tile_url)

In [9]:
tiles.head()

,quadkey,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices,geometry
0,1321222233020202,144116,125751,3,7330,1324,"POLYGON ((113.55469 22.16197, 113.56018 22.161..."
1,1230231303303002,89763,52939,42,726,137,"POLYGON ((55.41504 25.27947, 55.42053 25.27947..."
2,0313131210131030,62657,15479,19,271,40,"POLYGON ((-1.79077 51.55658, -1.78528 51.55658..."
3,0213330110110321,98061,11842,19,2,1,"POLYGON ((-93.22449 45.06576, -93.21899 45.065..."
4,3100130032202231,15167,7287,8,674,122,"POLYGON ((107.59460 -6.91552, 107.60010 -6.915..."


Get Boundaries files 

In [10]:
# zipfile of U.S. county boundaries
county_url = "https://www2.census.gov/geo/tiger/TIGER2019/COUNTY/tl_2019_us_county.zip" 
counties = gp.read_file(county_url)

In [11]:
counties.head()

,STATEFP,COUNTYFP,COUNTYNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,31,039,00835841,31039,Cuming,Cuming County,06,H1,G4020,None,None,None,A,1477652222,10690952,+41.9158651,-096.7885168,"POLYGON ((-97.01952 42.00410, -97.01952 42.004..."
1,53,069,01513275,53069,Wahkiakum,Wahkiakum County,06,H1,G4020,None,None,None,A,680962890,61582307,+46.2946377,-123.4244583,"POLYGON ((-123.43639 46.23820, -123.44759 46.2..."
2,35,011,00933054,35011,De Baca,De Baca County,06,H1,G4020,None,None,None,A,6016819475,29089486,+34.3592729,-104.3686961,"POLYGON ((-104.56739 33.99757, -104.56772 33.9..."
3,31,109,00835876,31109,Lancaster,Lancaster County,06,H1,G4020,339,30700,None,A,2169270569,22849484,+40.7835474,-096.6886584,"POLYGON ((-96.91075 40.78494, -96.91075 40.790..."
4,31,129,00835886,31129,Nuckolls,Nuckolls County,06,H1,G4020,None,None,None,A,1489645188,1718484,+40.1764918,-098.0468422,"POLYGON ((-98.27367 40.08940, -98.27367 40.089..."


In [12]:
# filter out the Kentucky fips code and reproject to match the tiles
ky_counties = counties.loc[counties['STATEFP'] == '21'].to_crs(4326)

In [13]:
ky_counties.head()

,STATEFP,COUNTYFP,COUNTYNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
9,21,053,00516873,21053,Clinton,Clinton County,06,H1,G4020,None,None,None,A,510875755,21152699,+36.7272577,-085.1360977,"POLYGON ((-85.23910 36.76983, -85.23443 36.786..."
58,21,003,00516848,21003,Allen,Allen County,06,H1,G4020,150,14540,None,A,891838779,19482100,+36.7507703,-086.1924580,"POLYGON ((-86.29580 36.85107, -86.29347 36.852..."
66,21,231,00516962,21231,Wayne,Wayne County,06,H1,G4020,None,None,None,A,1186690748,67283816,+36.8007715,-084.8265896,"POLYGON ((-85.04450 36.87093, -85.03908 36.874..."
133,21,209,00516951,21209,Scott,Scott County,06,H1,G4020,336,30460,None,A,729774435,9440820,+38.2857099,-084.5783405,"POLYGON ((-84.58054 38.47304, -84.58010 38.473..."
157,21,061,00516877,21061,Edmonson,Edmonson County,06,H1,G4020,150,14540,None,A,784468358,13258105,+37.2275136,-086.2180153,"POLYGON ((-86.39916 37.16990, -86.39932 37.170..."


Join tiles to counties 

In [14]:
tiles_in_ky_counties = gp.sjoin(tiles, ky_counties, how="inner", predicate='intersects')

In [15]:
# convert to Mbps for easier reading
tiles_in_ky_counties['avg_d_mbps'] = tiles_in_ky_counties['avg_d_kbps'] / 1000
tiles_in_ky_counties['avg_u_mbps'] = tiles_in_ky_counties['avg_u_kbps'] / 1000

In [16]:
tiles_in_ky_counties.head()

,quadkey,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices,geometry,index_right,STATEFP,COUNTYFP,...,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,avg_d_mbps,avg_u_mbps
370,0320003031132232,236452,55959,40,37,11,"POLYGON ((-85.85815 38.13888, -85.85266 38.138...",2534,21,111,...,350,31140,None,C,986032282,43964666,+38.1895321,-085.6576243,236.452,55.959
1653,0320003121002010,128046,59741,42,76,13,"POLYGON ((-85.41870 38.23818, -85.41321 38.238...",2534,21,111,...,350,31140,None,C,986032282,43964666,+38.1895321,-085.6576243,128.046,59.741
2825,0320003102323032,228717,68654,38,18,10,"POLYGON ((-85.55054 38.29425, -85.54504 38.294...",2534,21,111,...,350,31140,None,C,986032282,43964666,+38.1895321,-085.6576243,228.717,68.654
8662,0320003120131011,6940,999,458,8,2,"POLYGON ((-85.45715 38.20366, -85.45166 38.203...",2534,21,111,...,350,31140,None,C,986032282,43964666,+38.1895321,-085.6576243,6.940,0.999
9703,0320003031131230,154782,59655,35,23,8,"POLYGON ((-85.81421 38.17775, -85.80872 38.177...",2534,21,111,...,350,31140,None,C,986032282,43964666,+38.1895321,-085.6576243,154.782,59.655


Stat Summary

In [17]:
# def weighted_mean(group, avg_name, weight_name):
#     """ http://stackoverflow.com/questions/10951341/pandas-dataframe-aggregate-function-using-multiple-columns
#     In rare instance, we may not have weights, so just return the mean. Customize this if your business case
#     should return otherwise.
#     """
#     d = group[avg_name]
#     w = group[weight_name]
#     try:
#         return (d * w).sum() / w.sum()
#     except ZeroDivisionError:
#         return d.mean()
    
county_stats = (
    tiles_in_ky_counties.groupby(["GEOID", "NAMELSAD"])
    .apply(
        lambda x: pd.Series(
            {"avg_d_mbps_wt": np.average(x["avg_d_mbps"], weights=x["tests"])}
        )
    )
    .reset_index()
    .merge(
        tiles_in_ky_counties.groupby(["GEOID", "NAMELSAD"])
        .agg(tests=("tests", "sum"))
        .reset_index(),
        on=["GEOID", "NAMELSAD"],
    )
)

In [18]:
table_stats = (
    county_stats.loc[county_stats["tests"] >= 50]
    .nlargest(20, "avg_d_mbps_wt")
    .append(
        county_stats.loc[county_stats["tests"] >= 50].nsmallest(20, "avg_d_mbps_wt")
    )
    .sort_values("avg_d_mbps_wt", ascending=False)
    .round(2) # round to 2 decimal places for easier reading
)

In [19]:
header = ["GEOID", "County", "Avg download speed (Mbps)", "Tests"]

table_stats.rename(columns=dict(zip(table_stats.columns, header)))

,GEOID,County,Avg download speed (Mbps),Tests
55,21111,Jefferson County,159.02,72699
33,21067,Fayette County,154.13,50730
104,21209,Scott County,151.21,6632
79,21159,Martin County,150.30,1489
23,21047,Christian County,146.73,5365
92,21185,Oldham County,144.92,9787
38,21077,Gallatin County,144.21,673
75,21151,Madison County,144.17,6097
24,21049,Clark County,144.07,2525
6,21013,Bell County,143.69,1165
